In [1]:
%gui qt
%load_ext autoreload
%autoreload 2
    
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.Bnd import Bnd_OBB
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_Transform,
    BRepBuilderAPI_MakeEdge,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeFace
)
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE
from OCC.Core.TopoDS import TopoDS_Compound, TopoDS_Wire
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.gp import gp_Trsf, gp_Pnt, gp_Ax3, gp_Dir, gp_Pln, gp_Vec
from OCC.Display.SimpleGui import init_display
from IPython.display import display
from IPython.display import display, HTML

import numpy as np
import json

DSTV_FACE_MAP = {'I':['O','U','V'], 'U':['H','U','O'], 'L':['H','U']}

# Start display without blocking
viewer, start_display, add_menu, add_function_to_menu = init_display()

# JSON Library path
json_path = "../data/Shape_classifier_info.json"

viewer.EraseAll()
viewer.View_Iso()
viewer.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
def load_step(file_path: str):
    reader = STEPControl_Reader()
    status = reader.ReadFile(file_path)
    if status != 1:
        raise RuntimeError(f"Failed to read STEP file: {file_path}")
    reader.TransferRoots()
    return reader.Shape()

In [3]:
def visualize(shape, obb, display, color="CYAN", clear=True):
    """
    Displays a shape with global XYZ axes and OBB extents.
    
    Args:
        shape: The TopoDS_Shape to display.
        obb: The associated OBB object for size/scale reference.
        display: OCC display handle (e.g., from init_display()).
        color: Optional color for the shape.
        clear: If True, erase all previous shapes before displaying.
    """
    # if clear:
    #     viewer.EraseAll()

    # Display the shape
    viewer.DisplayShape(shape, color=color, update=False)

    # Draw global axes
    origin = gp_Pnt(0, 0, 0)
    scale = max(2 * obb.XHSize(), 2 * obb.YHSize(), 2 * obb.ZHSize()) * 0.5

    # Axis lines
    axes = [
        ((scale, 0, 0), "RED"),
        ((0, scale, 0), "GREEN"),
        ((0, 0, scale), "BLUE")
    ]

    for vec, axis_color in axes:
        end = gp_Pnt(*vec)
        edge = BRepBuilderAPI_MakeEdge(origin, end).Edge()
        viewer.DisplayShape(edge, color=axis_color, update=False)

    viewer.FitAll()



In [4]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def draw_obb_box(obb, viewer, color="ORANGE", update=True):
    """
    Draws the 12 edges of the OBB as lines in the viewer.
    """
    c = obb.Center()
    cx, cy, cz = c.X(), c.Y(), c.Z()
    hx, hy, hz = obb.XHSize(), obb.YHSize(), obb.ZHSize()
    dx, dy, dz = obb.XDirection(), obb.YDirection(), obb.ZDirection()

    # Precompute corner offsets
    offsets = [
        (+hx, +hy, +hz), (-hx, +hy, +hz), (-hx, -hy, +hz), (+hx, -hy, +hz),
        (+hx, +hy, -hz), (-hx, +hy, -hz), (-hx, -hy, -hz), (+hx, -hy, -hz)
    ]

    # Compute corners
    corners = []
    for ox, oy, oz in offsets:
        x = cx + ox * dx.X() + oy * dy.X() + oz * dz.X()
        y = cy + ox * dx.Y() + oy * dy.Y() + oz * dz.Y()
        z = cz + ox * dx.Z() + oy * dy.Z() + oz * dz.Z()
        corners.append(gp_Pnt(x, y, z))

    # Define edges by corner indices
    edges_idx = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # top face
        (4, 5), (5, 6), (6, 7), (7, 4),  # bottom face
        (0, 4), (1, 5), (2, 6), (3, 7),  # vertical edges
    ]

    for i1, i2 in edges_idx:
        edge = BRepBuilderAPI_MakeEdge(corners[i1], corners[i2]).Edge()
        viewer.DisplayShape(edge, color=color, update=False)

    if update:
        viewer.FitAll()

In [5]:
def compute_obb(shape):
    """
    Compute the Oriented Bounding Box for a shape.
    Returns a Bnd_OBB instance.
    """
    obb = Bnd_OBB()
    brepbndlib.AddOBB(shape, obb)
    return obb

In [6]:
def draw_obb_axes(obb, viewer, scale=500):
    """
    Visualize the X, Y, Z axes of the OBB in the viewer as arrows.
    """
    from OCC.Core.gp import gp_Pnt
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

    origin_xyz = obb.Center()
    origin = gp_Pnt(origin_xyz.X(), origin_xyz.Y(), origin_xyz.Z())

    axes = [
        (obb.XDirection(), "RED"),
        (obb.YDirection(), "GREEN"),
        (obb.ZDirection(), "BLUE"),
    ]

    for dir_vec, color in axes:
        end = gp_Pnt(
            origin.X() + scale * dir_vec.X(),
            origin.Y() + scale * dir_vec.Y(),
            origin.Z() + scale * dir_vec.Z()
        )
        edge = BRepBuilderAPI_MakeEdge(origin, end).Edge()
        viewer.DisplayShape(edge, color=color, update=False)

    viewer.FitAll()


In [7]:
if __name__ == "__main__":

    step_path = "../data/0444-1 ANGLED.step"
    # step_path = "../data/ncTest.step"
    # step_path = "../data/TestEA.step"
    # step_path = "../data/TestEAMirror.step"
    # step_path = "../data/TestUEA.step"
    # step_path = "../data/TestUEAMirror.step"
    # step_path = "../data/TestPFC.step"
    
    # Load shape and compute OBB once
    shape_orig = load_step(step_path)
    obb_orig = compute_obb(shape_orig)

    # visualize(shape_orig, obb_orig, viewer)
    visualize(shape_orig, obb_orig, viewer)
    draw_obb_box(obb_orig, viewer, color="ORANGE")
    draw_obb_axes(obb_orig, viewer, scale=500)
    


Many colors for color name CYAN, using first.
Many colors for color name BLUE, using first.
Many colors for color name BLUE, using first.
